This notebook shows how to import a DPFA model as a tensorflow keras model and train it.

# 1. System Setup

In [1]:
from google.colab import drive
import tensorflow as tf
import os
import sys

BASE_DIR = '/content/dirve'
CODE_DIR = os.path.join(BASE_DIR, 'My Drive')
DATA_DIR = os.path.join(CODE_DIR, 'dpfa', 'data')

In [2]:
# check if GPU is visible
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# mount google drive to access data
drive.mount(BASE_DIR, force_remount=True)

Mounted at /content/dirve


In [4]:
# change to the destination dir
os.chdir(os.path.join(BASE_DIR, 'MyDrive', 'dpfa', 'dpfa'))

In [5]:
# only run once at start
!pip install -r ../requirements.txt 

     |████████████████████████████████| 2.0 MB 12.8 MB/s 
     |████████████████████████████████| 829 kB 46.2 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 7.1 MB 28.9 MB/s 
     |████████████████████████████████| 152 kB 46.0 MB/s 
     |████████████████████████████████| 95 kB 4.3 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 10.3 MB 39.2 MB/s 
     |████████████████████████████████| 37.1 MB 80 kB/s 
     |████████████████████████████████| 10.8 MB 36.8 MB/s 
     |████████████████████████████████| 3.0 MB 45.5 MB/s 
     |████████████████████████████████| 296 kB 57.8 MB/s 
     |████████████████████████████████| 99 kB 9.5 MB/s 
     |████████████████████████████████| 247 kB 56.5 MB/s 
     |████████████████████████████████| 510 kB 48.5 MB/s 
     |████████████████████████████████| 636 kB 44.1 MB/s 
     |████████████████████████████████| 62 kB 946 kB/s 
     |█████████████████

In [5]:
# dpfa
import dpfa_config, preprocessing, model, losses, main

# 2. Load Data

In [6]:
dataset = 'assist2017'
if dataset == 'synthetic5':
    max_len = 50
else:
    max_len = dpfa_config.MAX_LEN

if dataset == 'nips2020':
    raise NotImplementedError
else:
    data_dir = dpfa_config.DATA_DIR

data = preprocessing.DataSet(
    dataset,   # name of dataset
    data_dir,  # data director
    0,         # cv idx 
    max_len=max_len, # max_len
    **dpfa_config.config[dataset]  # other default training parameters
    )

In [7]:
data.load_data()

train data in /content/dirve/MyDrive/dpfa/data/assist2017-cv-train-0.csv is loaded
test data in /content/dirve/MyDrive/dpfa/data/assist2017-cv-test-0.csv is loaded


In [8]:
data.describe_data()

dataset = assist2017
num of attempts = 393.663 K
num of students = 1709
num of items = 4119


In [9]:
(train_inputs, train_targets), (test_inputs, test_targets) = data.preprocess()

input dataset is raw, will perform encode and extract features
problem encoder is created
train and test data are encoded
train and test sequences are extracted
100%|██████████| 342/342 [00:00<00:00, 42254.39it/s]
train and test sequences are folded
train and test inputs and targets are created


# 3. Build Model

In [10]:
dpfa_model = model.DPFABase(
    hidden_size=16,
    dropout=0.2,
    item_vocab_size=data.item_vocab_size,
    regulate_dot_product=False,
    time_decay=False,
    normalize_embedding=True,
    )

In [11]:
batch_size = 512
lr = 0.008 * batch_size / 128 * max_len / 128

dpfa_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=losses.PaddedBinaryCrossentropyLoss(0.1)
)

In [12]:
dpfa_model.fit(
    train_inputs,
    train_targets,
    validation_split=0.1,
    epochs=60,
    batch_size=batch_size,
)

Epoch 1/60
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
4/4 [==============================] - 4s 101ms/step - loss: 0.7902 - val_loss: 0.7514
Epoch 2/60
4/4 [==============================] - 0s 28ms/step - loss: 0.7346 - val_loss: 0.7046
Epoch 3/60
4/4 [==============================] - 0s 27ms/step - loss: 0.6883 - val_loss: 0.6689
Epoch 4/60
4/4 [==============================] - 0s 27ms/step - loss: 0.6544 - val_loss: 0.6444
Epoch 5/60
4/4 [==============================] - 0s 27ms/step - loss: 0.6307 - val_loss: 0.6278
Epoch 6/60
4/4 [==============================] - 0s 28ms/step - loss: 0.6139 - val_loss: 0.6162
Epoch 7/60
4/4 [==============================] - 0s 27ms/step - loss: 0.6015 - val_loss: 0.6081
Epoch 8/60
4/4 [==============================] - 0s 27ms/step - loss: 0.5924 - val_loss: 0.6022
Epoch 9/60
4/4 [==============================] - 0s 28ms/step - loss: 0.5855 - val_l

In [13]:
# auc
main.evaluate(test_inputs, test_targets, dpfa_model, test_size=test_targets.shape[0])

0.7893170156788926